In [6]:
from sklearn.datasets import load_iris
import pandas as pd


In [8]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [10]:
df['target'] = iris.target_names[iris.target]
df


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [11]:
import numpy as np


In [12]:
y = np.array(df['target'].astype('category').cat.codes).astype(float)
X = np.array(df.iloc[:, :4])


In [14]:
from sklearn.model_selection import train_test_split
import torch


In [16]:
train_X, val_X, train_y, val_y = train_test_split(
    X, y, test_size=0.2, random_state=71)

train_X = torch.Tensor(train_X)
val_X = torch.Tensor(val_X)
train_y = torch.LongTensor(train_y)
val_y = torch.LongTensor(val_y)


In [19]:
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.manual_seed(71)


class DNN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


In [20]:
model = DNN()
print(model)


DNN(
  (fc1): Linear(in_features=4, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=3, bias=True)
)


In [27]:
from torch import optim

optimizer = optim.SGD(model.parameters(), lr=0.02)
train_loss = []
train_acc = []
i = 0

model.train()
for epoch in range(100):
    data, target = Variable(train_X), Variable(train_y)
    optimizer.zero_grad()
    output = model(data)

    loss = F.nll_loss(output, target)
    loss.backward()
    train_loss.append(loss.data.item())
    optimizer.step()

    prediction = output.data.max(1)[1]
    acc = prediction.eq(target.data).sum().numpy() / len(train_X)
    train_acc.append(acc)

    if i % 10 == 0:
        print(f'Train Step: {i} Loss: {loss.data.item()} Accuracy: {acc}')
    i += 1


print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(i, loss.data.item(), acc))

model.eval()
outputs = model(Variable(val_X))
_, predicted = torch.max(outputs.data, 1)
print(f'Acc: {round(predicted.eq(val_y).sum().numpy() / len(predicted), 3)}')


Train Step: 0 Loss: 0.1097024604678154 Accuracy: 0.975
Train Step: 10 Loss: 0.10774092376232147 Accuracy: 0.975
Train Step: 20 Loss: 0.10590311884880066 Accuracy: 0.975
Train Step: 30 Loss: 0.10417748242616653 Accuracy: 0.975
Train Step: 40 Loss: 0.10255272686481476 Accuracy: 0.975
Train Step: 50 Loss: 0.10102133452892303 Accuracy: 0.975
Train Step: 60 Loss: 0.09957613050937653 Accuracy: 0.9833333333333333
Train Step: 70 Loss: 0.09821265190839767 Accuracy: 0.9833333333333333
Train Step: 80 Loss: 0.09692239761352539 Accuracy: 0.9833333333333333
Train Step: 90 Loss: 0.09570158272981644 Accuracy: 0.9833333333333333
Train Step: 100	Loss: 0.095	Accuracy: 0.983
Acc: 0.967
